# Pairwise differential accessibility analysis and filtering
- Load and filter the new raw count matrix
- Assign cell type labels to samples
- Run DESeq2 for all pairwise cell type comparisons
- Identify differentially accessible regions specific to each cell type
- Filter for top 500 peaks per cell type by fold change and padj
- Exclude peaks overlapping Human Atlas immune, endothelial, and stromal modules
- Keep only selected cell types (BRCA, LUAD, LUSC, COAD, Hepatocytes)
- Merge new markers with existing ones
- Save final marker regions and region list as CSV and RDS files

## Load required libraries

In [ ]:
library(DESeq2)
library(dplyr)
library(tibble)
library(purrr)
library(BiocParallel)
library(parallel)
library(pbapply)
library(tidyr)
library(readr)

In [ ]:
# Register 8 cores for parallel processing
register(MulticoreParam(8))

## Load the raw counts with the new samples

In [ ]:
# Read counts from CSV
raw_counts <- read.csv("/mnt/DATA3/daniel/project/01_ATAC_preprocessing/data/raw_counts_matrix.csv",
                       check.names=FALSE)
# Construct a unique region name
raw_counts$region <- paste(raw_counts$chrom, raw_counts$start, raw_counts$end, sep="-")
rownames(raw_counts) <- raw_counts$region

# Remove columns we don't need
raw_counts <- raw_counts[ , !(colnames(raw_counts) %in% c("chrom","start","end","region"))]

# Round counts to integer
raw_counts <- round(raw_counts)


## Build samples conditions

In [ ]:
# Build sample_conditions
sample_conditions <- data.frame(
  cell_type = sapply(strsplit(colnames(raw_counts), "_"), `[`, 1),
  row.names = colnames(raw_counts)
)
sample_conditions$cell_type <- factor(sample_conditions$cell_type)

# filter out low-count rows
keep <- rowSums(raw_counts >= 10) >= 2
raw_counts_filtered <- raw_counts[keep, ]
cat("Filtered from", nrow(raw_counts), "to", nrow(raw_counts_filtered), "rows.\n")

## Create the DDS

In [ ]:
# Create DESeqDataSet
dds <- DESeqDataSetFromMatrix(
  countData = raw_counts_filtered,
  colData   = sample_conditions,
  design    = ~ cell_type
)

# Run DESeq (parallel=TRUE is optional, but can use more memory)
dds <- DESeq(dds, parallel=TRUE)

# Check available resultsNames
resultsNames(dds)

## Run pairwise comparison

In [ ]:
# Limit internal BLAS parallelism
Sys.setenv(OMP_NUM_THREADS="2")
Sys.setenv(MKL_NUM_THREADS="2")

# Define output directory
out_dir <- "/mnt/DATA3/daniel/project/04_DA_and_reference_building/data/pairwise_results"

# Ensure the output directory exists
if (!dir.exists(out_dir)) {
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)
}

# Define log file path
log_file <- file.path(out_dir, "run_log.txt")

# Initialize log file
writeLines(c("=== Pairwise DESeq2 Analysis Started ==="), log_file)

# Function to run one pairwise contrast (No Shrinkage)
save_contrast <- function(i, pairs, dds, out_dir) {
  ct1 <- pairs[i, 1]
  ct2 <- pairs[i, 2]
  
  # Print progress update
  cat("\n", Sys.time(), "- [", i, "/", nrow(pairs), "] Running contrast:", ct1, "vs", ct2, "\n")
  flush.console()

  # Recreate `dds` in each iteration to ensure independent execution
  dds_temp <- dds

  # Compute DE (No Shrinkage)
  res <- results(dds_temp, contrast=c("cell_type", ct1, ct2))

  # Convert to data frame
  df <- as.data.frame(res) %>%
    rownames_to_column("peak") %>%
    mutate(comparison = paste0(ct1, "_vs_", ct2))

  # Define output file path
  out_file <- file.path(out_dir, paste0(ct1, "_vs_", ct2, ".csv"))

  # Write results immediately to disk
  write.csv(df, out_file, row.names=FALSE)

  # Print completion message
  cat(Sys.time(), "- Finished writing:", out_file, "\n")
  flush.console()

  # Log progress
  log_msg <- paste(Sys.time(), "- Completed:", ct1, "vs", ct2, "- Memory BEFORE cleanup:", pryr::mem_used(), "bytes\n")
  write(log_msg, file = log_file, append = TRUE)

  # Force aggressive memory cleanup
  rm(dds_temp, res, df)
  invisible(capture.output(gc(full = TRUE)))

  # Log memory after cleanup
  log_msg <- paste(Sys.time(), "- Memory AFTER cleanup:", pryr::mem_used(), "bytes\n")
  write(log_msg, file = log_file, append = TRUE)
}

# Build list of cell-type pairs
all_ct <- levels(dds$cell_type)
pairs  <- t(combn(all_ct, 2))  # Generate all unique cell-type comparisons

# Track which contrasts have already been processed
completed_files <- list.files(out_dir, pattern = "*.csv")
completed_pairs <- sub(".csv", "", completed_files)

# Sequential execution with forced memory cleanup
for (i in seq_len(nrow(pairs))) {
  ct1 <- pairs[i, 1]
  ct2 <- pairs[i, 2]
  
  # Skip already completed pairs (prevents reprocessing on crashes)
  if (paste0(ct1, "_vs_", ct2) %in% completed_pairs) {
    cat("Skipping", ct1, "vs", ct2, "- Already processed\n")
    next
  }

  # Run contrast
  save_contrast(i, pairs, dds, out_dir)
}

cat("\nAll pairwise contrasts saved in", out_dir, "\n")
write("\n=== Pairwise DESeq2 Analysis Completed ===", file = log_file, append = TRUE)


## Filter pairwise results based on log2 fold change and padj

In [ ]:
# Parameters for calling specific peaks
lfc_cut  <- 1.5
padj_cut <- 0.1

# Output directories
results_dir <- "/mnt/DATA3/daniel/project/04_DA_and_reference_building/data/pairwise_results"
output_dir <- "/mnt/DATA3/daniel/project/04_DA_and_reference_building/data/celltype_specific_peaks"

# Ensure output directory exists
if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)
}

# Store specific peaks for each cell type
specific_peaks_list <- list()

# Function to check specific peaks for a given target cell type
check_specific_peaks <- function(target_ct, results_dir, total_cts) {
  others <- setdiff(all_ct, target_ct)
  keep_peaks <- NULL
  file_count <- 0
  
  cat("\n--- Processing:", target_ct, "(", which(all_ct == target_ct), "/", total_cts, ") ---\n")
  flush.console()
  
  for (ot in others) {
    comp_name <- paste0(target_ct, "_vs_", ot)
    rev_comp_name <- paste0(ot, "_vs_", target_ct)
    
    comp_file <- file.path(results_dir, paste0(comp_name, ".csv"))
    rev_file <- file.path(results_dir, paste0(rev_comp_name, ".csv"))
    
    if (file.exists(comp_file)) {
      df_tgt <- read.csv(comp_file) %>%
        filter(log2FoldChange >= lfc_cut, padj < padj_cut) %>%
        select(peak, log2FoldChange, padj)  # Keep relevant columns
    } else if (file.exists(rev_file)) {
      df_tgt <- read.csv(rev_file) %>%
        filter(log2FoldChange <= -lfc_cut, padj < padj_cut) %>%
        mutate(log2FoldChange = -log2FoldChange) %>%
        select(peak, log2FoldChange, padj)
    } else {
      cat("Warning: No file found for", target_ct, "vs", ot, "\n")
      flush.console()
      next
    }
    
    if (is.null(keep_peaks)) {
      keep_peaks <- df_tgt
    } else {
      # Merge results, keeping the peak with the lowest padj value
      keep_peaks <- inner_join(keep_peaks, df_tgt, by = "peak", suffix = c("", "_new")) %>%
        mutate(
          padj = pmin(padj, padj_new, na.rm = TRUE),  # Keep lowest padj
          log2FoldChange = pmax(log2FoldChange, log2FoldChange_new, na.rm = TRUE) # Keep highest log2FC
        ) %>%
        select(peak, log2FoldChange, padj)  # Keep only relevant columns
    }
    
    file_count <- file_count + 1
    cat("Checked:", target_ct, "vs", ot, "| Peaks remaining:", nrow(keep_peaks), "\n")
    flush.console()
  }
  
  cat("Total files checked for", target_ct, ":", file_count, "\n")
  flush.console()
  
  return(keep_peaks)
}

# Iterate over all cell types and find "specific" peaks
total_cts <- length(all_ct)
for (ct in all_ct) {
  cat("\n>>> Starting analysis for:", ct, "\n")
  flush.console()
  
  out_peaks <- check_specific_peaks(ct, results_dir, total_cts)
  
  if (!is.null(out_peaks) && nrow(out_peaks) > 0) {
    cat(ct, ":", nrow(out_peaks), "specific peaks found\n")
  } else {
    cat(ct, ": no specific peaks found\n")
  }
  flush.console()
  
  specific_peaks_list[[ct]] <- out_peaks
}

# Select the peak with the maximum log2FoldChange and lowest padj per region
specific_peaks_list <- lapply(specific_peaks_list, function(df) {
  if (!is.null(df) && nrow(df) > 0) {
    df %>%
      group_by(peak) %>%
      arrange(desc(log2FoldChange), padj) %>%  
      slice(1) %>%
      ungroup()
  } else {
    NULL
  }
})

# Save each set of specific peaks
for (ct in names(specific_peaks_list)) {
  out_df <- specific_peaks_list[[ct]]
  if (!is.null(out_df) && nrow(out_df) > 0) {
    write.csv(
      out_df, 
      file.path(output_dir, paste0(ct, "_specific_peaks.csv")),
      row.names=FALSE
    )
    cat("Saved:", ct, "with", nrow(out_df), "specific peaks\n")
  }
  flush.console()
}

# Final summary
cat("\n==== SUMMARY OF SPECIFIC PEAKS ====\n")
flush.console()
for (ct in names(specific_peaks_list)) {
  cat(ct, ":", ifelse(is.null(specific_peaks_list[[ct]]), 0, nrow(specific_peaks_list[[ct]])), "specific peaks\n")
  flush.console()
}

cat("\nDone. Check", output_dir, "for results.\n")
flush.console()


## Load specific peaks again into memory if necessary

In [ ]:
# Define the directory where files are saved
# output_dir <- "/mnt/DATA3/daniel/project/04_DA_and_reference_building/data/celltype_specific_peaks"

# List all CSV files in the directory
# peak_files <- list.files(output_dir, pattern = "*.csv", full.names = TRUE)

# Read each file into a list, using the filename
# specific_peaks_list <- lapply(peak_files, function(file) {
  # read.csv(file) %>% as_tibble()
# })

# Name the list elements based on file names 
# names(specific_peaks_list) <- gsub("_specific_peaks.csv", "", basename(peak_files))

# Check the structure of the reloaded data
# str(specific_peaks_list)


# Sort log2 fold change and padj

In [ ]:
# Ensure correct sorting: highest log2FoldChange first, then lowest padj
specific_peaks_list <- lapply(specific_peaks_list, function(df) {
  if (!is.null(df) && nrow(df) > 0) {
    df %>%
      arrange(desc(log2FoldChange), padj)  
  } else {
    NULL
  }
})


In [ ]:
# Split the "peak" column into chr, start, and end
split_peak_column <- function(df) {
  if (is.null(df) || nrow(df) == 0) return(df)
  
  # Check if peak column exists
  if ("peak" %in% colnames(df)) {
    df <- df %>%
      separate(peak, into = c("chr", "start", "end"), sep = "[:-]", convert = TRUE)
  }
  
  return(df)
}

specific_peaks_list <- lapply(specific_peaks_list, split_peak_column)

# Human tissue atlas

In [ ]:
# Load the Human Tissue Atlas
human_atlas_path <- "/mnt/DATA3/daniel/project/01_ATAC_preprocessing/data/gabriel/markers_identification_input_files/Human_Atlas_peaks.txt"

human_atlas <- read_tsv(human_atlas_path, col_types = cols())

# Ensure numeric conversion for start and end positions
human_atlas <- human_atlas %>%
  mutate(chr = as.character(chr), start = as.numeric(start), end = as.numeric(end))

In [ ]:
# Define excluded modules
immune_modules <- 8:25
endothelial_modules <- 26:35
stromal_modules <- c(41:49, 139:150)
excluded_modules <- c(immune_modules, endothelial_modules, stromal_modules)

# Filter out Human Atlas peaks belonging to excluded modules
excluded_human_atlas <- human_atlas %>%
  filter(cCDRE_module %in% excluded_modules)

# Filter out ChrX and ChrY, selecto top 500 based on highest log2 fold change and lowest padj and filter for overlap in the human tissue atlas

In [ ]:
# Function to filter peaks based on overlap
filter_overlapping_peaks <- function(df, atlas_peaks, overlap_threshold) {
  if (is.null(df) || nrow(df) == 0) return(df)
  
  df <- df %>%
    rowwise() %>%
    filter(!any(
      (chr == atlas_peaks$chr) & 
      (pmin(end, atlas_peaks$end) - pmax(start, atlas_peaks$start)) / (end - start) >= overlap_threshold
    )) %>%
    ungroup()
  
  return(df)
}

# Select top 500 peaks before applying filtering
top_500_da_regions <- list()
for (cell_type in names(specific_peaks_list)) {
  
  df <- specific_peaks_list[[cell_type]]
  
  if (!is.null(df) && nrow(df) > 0) {
    df <- df %>%
      filter(!(chr %in% c("chrX", "chrY"))) %>%  
      arrange(desc(log2FoldChange), padj) %>%  
      slice_head(n = 500)  
    
    top_500_da_regions[[cell_type]] <- df
  }
}

# Apply filtering after selecting the top 500 peaks
filtered_top_500_da_regions <- list()
for (cell_type in names(top_500_da_regions)) {
  
  df <- top_500_da_regions[[cell_type]]
  
  # Remove peaks associated with excluded modules
  if ("cCDRE_module" %in% colnames(df)) {
    df <- df %>% filter(!(cCDRE_module %in% excluded_modules))
  }
  
  # Remove peaks overlapping the excluded Human Atlas peaks by 15%
  df <- filter_overlapping_peaks(df, excluded_human_atlas, overlap_threshold = 0.15)
  
  # Remove peaks overlapping **any** Human Atlas peak by 60%
  df <- filter_overlapping_peaks(df, human_atlas, overlap_threshold = 0.60)

  # Store final filtered and sorted results
  filtered_top_500_da_regions[[cell_type]] <- df
}

# Print summary of filtered peaks
cat("\n==== FILTERED PEAKS SUMMARY ====\n")
for (cell_type in names(filtered_top_500_da_regions)) {
  cat(cell_type, "=> final # of DA peaks:", nrow(filtered_top_500_da_regions[[cell_type]]), "\n")
}


# Select only the new cell type markers

In [ ]:
# Define the cell types to keep
new_cell_types <- c("BRCA", "hepatocytes", "COAD", "LUAD", "LUSC")


In [ ]:
# Filter the top 500 DA regions to keep only selected cell types
filtered_top_500_selected <- list()
for (cell_type in new_cell_types) {
  if (cell_type %in% names(filtered_top_500_da_regions)) {
    
    df <- filtered_top_500_da_regions[[cell_type]]
    
    # Rename hepatocytes to Hepatocytes
    new_name <- ifelse(cell_type == "hepatocytes", "Hepatocytes", cell_type)
    
    # Ensure the dataframe contains the necessary columns before proceeding
    if (!all(c("chr", "start", "end", "log2FoldChange", "padj") %in% colnames(df))) {
      stop(paste("Error: Missing required columns in", new_name))
    }

    filtered_top_500_selected[[new_name]] <- df
  }
}



In [ ]:
# Convert peaks into "chr:start-end" format
df_filtered_regions <- lapply(filtered_top_500_selected, function(df) {
  if (!is.null(df) && nrow(df) > 0) {
    df %>%
      mutate(region = paste0(chr, ":", start, "-", end)) %>%
      select(region)  
  } else {
    NULL
  }
})



In [ ]:
# Print example regions from one selected cell type
if (length(df_filtered_regions) > 0) {
  example_cell_type <- names(df_filtered_regions)[1]  
  cat("\nExample regions for", example_cell_type, ":\n")
  print(head(df_filtered_regions[[example_cell_type]], 5))  
}


# Load the original markers and merge with new markers

In [ ]:
# Load old marker peaks
marker_peaks_path <- "/mnt/DATA3/daniel/project/02_cfDNA_preprocessing/data/cell_types_markers.csv"

marker_peaks <- read.csv(marker_peaks_path)

In [ ]:
# Convert new markers into correct format and merge with old ones
new_markers_list <- list()
for (cell_type in names(df_filtered_regions)) {
  for (region in df_filtered_regions[[cell_type]]$region) {
    parts <- unlist(strsplit(region, "[:-]"))
    if (length(parts) == 3) {
      new_markers_list <- append(new_markers_list, list(data.frame(
        cell_type = cell_type,
        chrom = parts[1],
        start = as.integer(parts[2]),
        end = as.integer(parts[3])
      )))
    }
  }
}

new_markers_df <- do.call(rbind, new_markers_list)


In [ ]:
# Merge old and new marker peaks
merged_markers_df <- bind_rows(marker_peaks, new_markers_df) %>% distinct()

# Remove duplicates
merged_markers_df <- merged_markers_df %>% distinct(chrom, start, end, .keep_all = TRUE)

# Final output summary
head(merged_markers_df)
cat("Total unique markers:", nrow(merged_markers_df), "\n")

# Save new cell type marker list as csv

In [ ]:
# Define output file path
output_file <- "/mnt/DATA3/daniel/project/04_DA_and_reference_building/data/new_pairwise_cell_types_markers.csv"

# Save dataframe to CSV file with the same structure as shown
write.csv(merged_markers_df, output_file, row.names = FALSE, quote = FALSE)

cat("File saved successfully at:", output_file, "\n")


# Create and save sigPeaks_pairwise and markers_pairwise

In [ ]:
# Create sigPeaks_pairwise
merged_markers_df <- merged_markers_df %>%
  mutate(region = paste0(chrom, "-", start, "-", end))  

sigPeaks_pairwise <- unique(merged_markers_df$region)  

# Create markers_pairwise 
markers_pairwise <- split(merged_markers_df$region, merged_markers_df$cell_type)

# Print to verify structure
cat("\n Structure of sigPeaks_pairwise:\n")
print(head(sigPeaks_pairwise, 5))  

cat("\n Structure of markers_pairwise:\n")
print(head(markers_pairwise, 5))  

# Define output paths for two separate RDS files
output_dir <- "/mnt/DATA3/daniel/project/04_DA_and_reference_building/data/"

sigPeaks_rds_path <- file.path(output_dir, "sigPeaks_pairwise.rds")
markers_rds_path <- file.path(output_dir, "markers_pairwise.rds")

# Save each objects
saveRDS(sigPeaks_pairwise, file = sigPeaks_rds_path)
saveRDS(markers_pairwise, file = markers_rds_path)

# Final output message
cat("Saved sigPeaks_pairwise to:", sigPeaks_rds_path, "\n")
cat("Saved markers_pairwise to:", markers_rds_path, "\n")
